In [47]:
import numpy as np 
import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [48]:
#Cargamos el csv

events = pd.read_csv('../events.csv')

## Analisis visita + checkout + conversion

In [4]:
# aca podemos analizar por que por la url podemos saber si e suna venta o ocmpra
events['operacion']= events['url'].str.split('/').str[1]
events['operacion'].value_counts()

comprar               54209
                      28323
vender                  221
clube-trocafone           2
termos-e-condicoes        1
Name: operacion, dtype: int64

In [49]:
#separo en tablas por eventos
viewed_product = events[events['event'] == 'viewed product']
brand_listing  = events[events['event'] == 'brand listing']
visited_site   = events[events['event'] == 'visited site']
ad_campaign_hit  = events[events['event'] == 'ad campaign hit']
searched_products  = events[events['event'] == 'searched products']
search_engine_hit  = events[events['event'] == 'search engine hit']
checkout  = events[events['event'] == 'checkout']
staticpage = events[events['event'] == 'staticpage']
conversion = events[events['event'] == 'conversion']
lead = events[events['event'] == 'lead']


In [50]:
#events[events['event'] == 'conversion']['channel'].value_counts()

Series([], Name: channel, dtype: int64)

In [51]:
# Esta funcion toma la tabla y borra las columnas que tiene todas NAN par asu mejor estudio
def borrar_columnas_vacias(tabla):
    for columna in tabla.columns:
        if(tabla[columna].count() == 0):
            tabla = tabla.drop([columna],axis = 1)
    return tabla
            

In [99]:
#borrar_columnas_vacias(visited_site)

In [100]:
visited = borrar_columnas_vacias(visited_site)
visited = visited.drop_duplicates(subset = ['person'])
visited = visited.set_index('person')
conver = conversion.loc[:,['timestamp','sku','person']]
conver = conver.set_index(['person','sku'])
check = checkout.loc[:,['timestamp','sku','person']]
check = check.set_index(['person','sku'])
viewed = viewed_product.loc[:,['timestamp','sku','person','model','condition','storage','color']]
viewed = viewed.set_index(['person','sku'])
a = check.join(conver,how = 'inner',lsuffix=' de checkout',rsuffix=' de conversion')
b = a.join(viewed,how = 'inner',rsuffix=' de viewed')
b = b.reset_index().set_index('person')

b = b.drop_duplicates(subset = ['timestamp de conversion'])
b = b.drop_duplicates(subset = ['timestamp de checkout'])
b = b.drop_duplicates(subset = ['timestamp'])
#aca puedo agregarle Pais,ciudad y region  jjoineando con la tabla  visited_site
c = visited.join(b,how = 'right',lsuffix=' de visited')
c = c.reset_index().set_index(['person','sku'])
c = c.rename(columns = {"timestamp":"timestamp de viewed"})
#c.loc[:,['timestamp de visited','timestamp de viewed','timestamp de checkout','timestamp de conversion','model','condition','storage','color','country']]
c.loc[:,['timestamp de viewed','timestamp de checkout','timestamp de conversion','model','condition','storage','color','country']]



,,timestamp de viewed,timestamp de checkout,timestamp de conversion,model,condition,storage,color,country
person,sku,,,,,,,,
00c13dee,6650.0,2018-04-26 22:13:37,2018-04-26 22:15:27,2018-04-26 22:28:53,Samsung Galaxy Core Plus Duos TV,Muito Bom,4GB,Branco,Brazil
00fdbb4b,3348.0,2018-06-11 01:47:23,2018-06-11 01:47:34,2018-06-10 14:37:50,Samsung Galaxy S6 Flat,Muito Bom,32GB,Branco,Brazil
0146a9df,2694.0,2018-03-06 01:37:22,2018-03-16 13:41:36,2018-03-16 13:50:25,iPhone 5s,Bom,32GB,Cinza espacial,Brazil
01db2fe6,6357.0,2018-02-09 21:50:07,2018-02-09 21:54:43,2018-02-09 22:07:07,Samsung Galaxy J5,Bom,16GB,Preto,Brazil
03bdcd81,3301.0,2018-01-09 16:39:42,2018-01-09 16:44:17,2018-01-09 16:47:50,Samsung Galaxy S6 Edge,Excelente,32GB,Dourado,Brazil
04850add,2830.0,2018-01-22 16:37:57,2018-01-23 22:21:19,2018-01-23 22:26:36,iPhone 6,Bom,16GB,Cinza espacial,Unknown
04fbc241,6370.0,2018-01-10 18:41:29,2018-01-10 18:47:59,2018-01-10 18:59:05,Samsung Galaxy J5,Muito Bom,16GB,Dourado,Brazil
0562e9d2,88.0,2018-04-30 15:56:33,2018-04-30 15:57:32,2018-04-30 16:00:50,iPhone 4G,Excelente,16GB,Preto,Brazil
0585daeb,13248.0,2018-05-03 13:50:18,2018-05-03 13:50:57,2018-05-14 18:47:28,Samsung Galaxy S8 Plus,Bom,128GB,Preto,Brazil


In [68]:
conersion.

SyntaxError: invalid syntax (<ipython-input-68-2887af7d4f1d>, line 1)

In [ ]:
673